In [1]:
from math import log

In [2]:
# 计算给定数据集的香农熵
def calcShannonEnt(dataset):
    numberEntries = len(dataset)
    labelCounts = {}
    for featVec in dataset:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numberEntries
        shannonEnt -= prob * log(prob, 2)
    return shannonEnt

In [3]:
# 创建数据集
def createDataSet():
    dataSet = [[1, 1, 'yes']
              ,[1, 1, 'yes']
              ,[1, 0, 'no']
              ,[0, 1, 'no']
              ,[0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

In [4]:
myDat, labels = createDataSet()

In [5]:
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [6]:
calcShannonEnt(myDat)

0.9709505944546686

In [7]:
# 熵表示信息的不确定程度. 熵越高, 则混合的数据也越多, 可以在数据集中添加更多的分类, 观察熵是如何变化的
myDat[0][-1] = 'maybe'

In [8]:
myDat

[[1, 1, 'maybe'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [9]:
calcShannonEnt(myDat)

1.3709505944546687

In [10]:
#根据特征值划分数据集
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [11]:
myDat, labels = createDataSet()

In [12]:
splitDataSet(myDat, 0, 1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [13]:
splitDataSet(myDat, 0, 0)

[[1, 'no'], [1, 'no']]

In [14]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1 
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = i  
    return bestFeature

In [15]:
chooseBestFeatureToSplit(myDat)

0

In [16]:
# 获取频数最多的类别
import operator
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    sortedclassCount = sorted(iter(classCount.items()), key=operator.itemgetter(1), reverse=True)
    return sortedclassCount[0][0]

In [17]:
# 递归创建树
def createTree(dataSet, labels):
    # 获取数据集中的所有类标签
    classList = [example[-1] for example in dataSet]
    # 若数据集中只存在一个类, 则停止递归
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    # 若全部特征都遍历完, 则停止递归
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    # 选择最优的特征
    bestFeature = chooseBestFeatureToSplit(dataSet)
    print('bestFeature: %s'%bestFeature)
    bestFeatLabel = labels[bestFeature]
    # 构造树
    myTree = {bestFeatLabel: {}}
    del(labels[bestFeature])
    # 获取该特征下所有的类别
    featValues = [example[bestFeature] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeature, value), subLabels)
    return myTree
    

In [50]:
myDat, labels = createDataSet()

In [40]:
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [51]:
import copy
labels_dp = copy.deepcopy(labels)

In [52]:
myTree=createTree(myDat, labels)

bestFeature: 0
bestFeature: 0


In [56]:
# 使用决策树进行分类
def classify(inputTree, featLabels, testVec):
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if type(secondDict[key]).__name__ == 'dict':
                classLabel = classify(secondDict[key], featLabels, testVec)
            else:    classLabel = secondDict[key]
    return classLabel

In [47]:
myTree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

In [48]:
list(myTree.keys())[0]

'no surfacing'

In [53]:
labels

['flippers']

In [54]:
labels_dp

['no surfacing', 'flippers']

In [58]:
classify(myTree, labels_dp, [0,1])

'no'

In [63]:
myTree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

In [67]:
# 使用pickle模块存储决策树
def storeTree(inputTree, filename):
    import pickle
    fw = open(filename, 'wb')
    pickle.dump(inputTree, fw)
    fw.close()
    
def grabTree(filename):
    import pickle
    fr = open(filename, 'rb+')
    return pickle.load(fr)

In [65]:
storeTree(myTree, 'DT')

In [68]:
grabTree('DT')

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

In [74]:
pwd

'D:\\develop\\project\\machinelearninginaction3x\\Ch03'

In [82]:
# 使用决策树预测隐形眼镜类型
fileDir = 'D:\\develop\\project\\machinelearninginaction3x\\Ch03'
dataPath = os.path.join(fileDir, 'lenses.txt')
fr = open(dataPath)
lenses = [line.strip().split('\t') for line in fr.readlines()]

In [83]:
lenses

[['young', 'myope', 'no', 'reduced', 'no lenses'],
 ['young', 'myope', 'no', 'normal', 'soft'],
 ['young', 'myope', 'yes', 'reduced', 'no lenses'],
 ['young', 'myope', 'yes', 'normal', 'hard'],
 ['young', 'hyper', 'no', 'reduced', 'no lenses'],
 ['young', 'hyper', 'no', 'normal', 'soft'],
 ['young', 'hyper', 'yes', 'reduced', 'no lenses'],
 ['young', 'hyper', 'yes', 'normal', 'hard'],
 ['pre', 'myope', 'no', 'reduced', 'no lenses'],
 ['pre', 'myope', 'no', 'normal', 'soft'],
 ['pre', 'myope', 'yes', 'reduced', 'no lenses'],
 ['pre', 'myope', 'yes', 'normal', 'hard'],
 ['pre', 'hyper', 'no', 'reduced', 'no lenses'],
 ['pre', 'hyper', 'no', 'normal', 'soft'],
 ['pre', 'hyper', 'yes', 'reduced', 'no lenses'],
 ['pre', 'hyper', 'yes', 'normal', 'no lenses'],
 ['presbyopic', 'myope', 'no', 'reduced', 'no lenses'],
 ['presbyopic', 'myope', 'no', 'normal', 'no lenses'],
 ['presbyopic', 'myope', 'yes', 'reduced', 'no lenses'],
 ['presbyopic', 'myope', 'yes', 'normal', 'hard'],
 ['presbyopic', 

In [84]:
lensesLabels = ['age', 'prescript', 'astigmatic', 'tearRate']

In [85]:
lensesTree = createTree(lenses, lensesLabels)

bestFeature: 3
bestFeature: 2
bestFeature: 1
bestFeature: 0
bestFeature: 0
bestFeature: 0
